In [ ]:
!pip install neo4j openai

In [ ]:
import json
from pathlib import Path

In [ ]:


sample_dic = {"a":1, "b" : {2}}

def reformat_dict(dict):
  for value, item in dict.items():
      if not isinstance(item, (int,str)):
        dict[value] = str(item)
      else:
        pass
  return dict

print(reformat_dict(sample_dic))



In [ ]:
# ## Standardise the data

# std_data = {}
# with open("/Users/rahul.kumar23/Drive/outcomes-ai-assignment/patients/patient_1.json", 'r') as f:
#     data = json.load(f)
#     for node in data:
#         node_name = list(node.keys())[0]
#         if node_name == "patient":
#             for key, value in node.items():
#                 std_data[key] = str(value)
        
#         else:
#             print("processing " , node_name, type(node[node_name]))
    
#             for dic_item in node[node_name]:
#                 for key,value in item.dic_items():
#                     # new_data[]
                

# print(std_data)

In [ ]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def execute_query(self, query, parameters=None):
        with self._driver.session() as session:
            return session.write_transaction(lambda tx: tx.run(query, parameters).data())

# Initialize Neo4j Connection
def connect_neo4j():
    uri = "bolt://localhost:7687"  # Default URI for Neo4j
    user = "neo4j"
    password = "test1234"  # Replace with your password
    return Neo4jConnection(uri, user, password)

In [ ]:
# data

In [ ]:
def store_in_neo4j(data, neo4j_conn):
    # Create Patient Node
    patient_query = """
    MERGE (p:Patient {patient_id: $patient_id})
    SET p.name = $name, p.dob = $dob, p.gender = $gender, p.address = $address, p.phone = $contact.phone, p.email = $contact.email
    """
    neo4j_conn.execute_query(patient_query, data["patient"])

In [ ]:
neo4j_connection = connect_neo4j()

In [ ]:
data = {"patient":{
    "patient_id": "p_002",
    "name": {
      "given": "Karen",
      "family": "Wee"
    },
    "gender": "female",
    "birthDate": "1985-04-12",
    "contact": [
      {
        "system": "phone",
        "value": "+1-555-123-4567",
        "use": "mobile"
      },
      {
        "system": "email",
        "value": "karen.wee@example.com",
        "use": "home"
      }
    ],
    "address": {
      "line": ["123 Maple Street"],
      "city": "Springfield",
      "state": "IL",
      "postalCode": "62704",
      "country": "USA"
    }
  }}

data = data["patient"]
for key,value in data.items():
    if not isinstance(value, str):
      print("got it")
      data[key] = str(data[key])

print(data)

In [ ]:
data = {"patient":{
    "patient_id": "p_002",
    "name": {
      "given": "Karen",
      "family": "Wee"
    },
    "gender": "female",
    "birthDate": "1985-04-12",
    "contact": [
      {
        "system": "phone",
        "value": "+1-555-123-4567",
        "use": "mobile"
      },
      {
        "system": "email",
        "value": "karen.wee@example.com",
        "use": "home"
      }
    ],
    "address": {
      "line": ["123 Maple Street"],
      "city": "Springfield",
      "state": "IL",
      "postalCode": "62704",
      "country": "USA"
    }
  }}

store_in_neo4j(data, neo4j_connection)

In [ ]:
with open("/Users/rahul.kumar23/Drive/outcomes-ai-assignment/patients/patient_2.json", 'r') as f:
    data = json.load(f)

In [ ]:
store_in_neo4j(data, neo4j_connection)

In [ ]:
import json
from pathlib import Path

# Function to create nodes and relationships in Neo4j
def store_in_neo4j(data, neo4j_conn):


    for node in data:
        node_name = list(node.keys())[0]

        if node_name == "patient":
            dict = reformat_dict(node[node_name])
            # print(dict)
            

            # Create Patient Node
            patient_query = """
            MERGE (p:Patient {patient_id: $patient_id})
            SET p.name = $name, p.dob = $dob, p.gender = $gender, p.address = $address, p.contact = $contact
            """
            neo4j_conn.execute_query(patient_query, dict)

    # Create Related Nodes and Relationships
        elif node_name=="encounters":
            
            for encounter in node["encounters"]:
                dict = reformat_dict(encounter)
                encounter_query = """
                MERGE (e:Encounter {id: $id})
                SET e.status = $status, e.type = $type, e.reasonCode = $reasonCode
                MERGE (p:Patient {patient_id: $patientId})
                MERGE (p)-[:HAS_ENCOUNTER]->(e)
                """
                neo4j_conn.execute_query(encounter_query, dict)
                print("added encounters")

    # for visit in data["visits"]:
    #     visit_query = """
    #     MERGE (v:Visit {id: $id})
    #     SET v.status = $status, v.type = $type, v.reasonCode = $reasonCode
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_VISIT]->(v)
    #     """
    #     neo4j_conn.execute_query(visit_query, visit)

    # for procedure in data["procedures"]:
    #     procedure_query = """
    #     MERGE (pr:Procedure {id: $id})
    #     SET pr.code = $code, pr.status = $status
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_PROCEDURE]->(pr)
    #     """
    #     neo4j_conn.execute_query(procedure_query, procedure)

    # for diagnosis in data["diagnosis"]:
    #     diagnosis_query = """
    #     MERGE (d:Diagnosis {id: $id})
    #     SET d.code = $code, d.notes = $notes
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_DIAGNOSIS]->(d)
    #     """
    #     neo4j_conn.execute_query(diagnosis_query, diagnosis)

    # for lab in data["labs"]:
    #     lab_query = """
    #     MERGE (l:Lab {code: $code})
    #     SET l.name = $labName, l.value = $value, l.referenceRange = $referenceRange
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_LAB]->(l)
    #     """
    #     neo4j_conn.execute_query(lab_query, lab)

    # for medication in data["medications"]:
    #     medication_query = """
    #     MERGE (m:Medication {code: $code})
    #     SET m.name = $genericName, m.dosage = $dosage, m.frequency = $frequency
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_MEDICATION]->(m)
    #     """
    #     neo4j_conn.execute_query(medication_query, medication)

    # for allergy in data["allergies"]:
    #     allergy_query = """
    #     MERGE (a:Allergy {id: $id})
    #     SET a.allergen = $allergenname, a.category = $category, a.reaction = $reaction
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_ALLERGY]->(a)
    #     """
    #     neo4j_conn.execute_query(allergy_query, allergy)

    # for note in data["notes"]:
    #     note_query = """
    #     MERGE (n:Note {id: $id})
    #     SET n.document_name = $document_name, n.entered_date = $entered_date
    #     MERGE (p:Patient {patient_id: $patient_id})
    #     MERGE (p)-[:HAS_NOTE]->(n)
    #     """
    #     neo4j_conn.execute_query(note_query, note)




In [ ]:
# Main Function to Parse JSON Files and Store in Neo4j
def parse_and_store_in_neo4j(json_dir, neo4j_conn):
    for file in Path(json_dir).glob("*.json"):
        with open(file, 'r') as f:
            data = json.load(f)
            store_in_neo4j(data, neo4j_conn)

if __name__ == "__main__":
    json_directory = "path_to_patient_json_files"
    neo4j_connection = connect_neo4j()
    parse_and_store_in_neo4j(json_directory, neo4j_connection)
    neo4j_connection.close()

In [119]:
import json
from openai import OpenAI


def invoke_llm(messages, cloud="sambanova", temperature=0.7):

    if cloud=="openai":
        api_key = "COYA AND FY"
        model = "gpt-4o-mini"
    elif cloud=="sambanova":
        model='Meta-Llama-3.1-70B-Instruct'
        api_key="f10708c4-f9dc-499c-b90f-eeec5c17e630"
        base_url="https://api.sambanova.ai/v1"

    client = OpenAI(
        api_key=api_key,
        base_url=base_url,
    )
    output = client.chat.completions.create(
                model = model,
                messages = messages, temperature=temperature, max_tokens = 1000, top_p=0.9, stream = False,
            )

    response = output.choices[0].message.content

    return response

In [140]:
with open("/Users/rahul.kumar23/Drive/outcomes-ai-assignment/patients/patient_1.json", 'r') as f:
    data = json.load(f)

patient_data = {}



for node in data:
    node_name = list(node.keys())[0]

    if node_name == "patient":
        print(node)
        patient_data["meta_data"]  = node['patient']

    else:
        # print(node)
        for idx,item in enumerate(node[node_name]):
            # print(item.values())
            system_prompt = f"you are a physician and reviews patient records before consultations. Given below is data for {node_name} of a patient. Your task is to summarize this data in plain english. Just return summarized important data and drop the non-usefull details. Keep the summary short and informative."
            prompt = item.values()
            messages = [
                {
                    "role": "system",
                    "content": f"{system_prompt}"
                },
                {
                    "role":"user",
                    "content" : prompt
                }
                    ]
            # print(messages)
            summary = invoke_llm(messages)
            patient_data[f"{node_name}_{idx}"] = summary
            import time
            time.sleep(10)
            # break
   

{'patient': {'patient_id': 'p_001', 'name': 'John A. Doe', 'dob': '1965-04-12', 'gender': 'male', 'address': '456 Elm Street, Springfield, USA', 'contact': {'phone': '555-123-4567', 'email': 'johndoe@example.com'}}}


In [141]:
patient_data

{'meta_data': {'patient_id': 'p_001',
  'name': 'John A. Doe',
  'dob': '1965-04-12',
  'gender': 'male',
  'address': '456 Elm Street, Springfield, USA',
  'contact': {'phone': '555-123-4567', 'email': 'johndoe@example.com'}},
 'encounters_0': '**Patient Encounter Summary:**\n\n- **Encounter Type:** Emergency Department Visit\n- **Date:** March 15, 2021\n- **Symptoms:** Shortness of breath\n- **Diagnosis:** Acute Exacerbation of Chronic Obstructive Pulmonary Disease\n- **Attending Physician:** Dr. Emily Richards\n- **Location:** Springfield General Hospital, Emergency Room 1\n- **Status:** Completed and billed ($750)',
 'encounters_1': '**Patient Encounter Summary**\n\n- **Encounter Type:** Outpatient\n- **Reason for Visit:** Follow-up Visit for Lung Cancer\n- **Diagnosis:** Stage II Non-Small Cell Lung Cancer\n- **Attending Physician:** Dr. Michael Thompson\n- **Location:** Oncology Outpatient Clinic, National Cancer Center\n- **Date:** April 10, 2021\n- **Procedure:** Office visit f

In [142]:
# print(invoke_llm(messages))

In [143]:
file_path = "/Users/rahul.kumar23/Drive/outcomes-ai-assignment/patients/patient_1_cleaned.json"
with open(file_path, 'w') as json_file:
    json.dump(patient_data, json_file, indent=4) 